In [3]:
#pip install pandas


In [4]:
#pip install matplotlib

In [4]:
pip install openpyxl

   ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/250.0 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------  245.8/250.0 kB 3.8 MB/s eta 0:00:01
   ---------------------------------------- 250.0/250.0 kB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import xpress as xp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Create a problem 

stage_1 = xp.problem(name='F_O stage 1')

Using the license file found in your Xpress installation. If you want to use this license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('C:/xpressmp/bin/xpauth.xpr')


In [58]:
# Import data into panda dataframes

df_rooms = pd.read_excel('Relevant Data Room.xlsx', header=0)
df_course_demand = pd.read_excel('Course Data (1).xlsx', header=0, sheet_name = 'Dict')[['Size']]
df_courses = pd.read_excel('Course Data (1).xlsx', header=0, sheet_name='Dict')
df_curricula = pd.read_excel('Baseline solution(ACHIEVED DELIVERABLE) and extracted data(ONGOING).xlsx', header=0, sheet_name='Curricula')

df_rooms_types = pd.read_excel('Relevant Data Room.xlsx', header=0, sheet_name='ROOM_DIC')[['ROOM INDEX', 'ROOM NAME', 'CAP', 'ROOM LAYOUT']]
#df_courses

In [59]:
# Index sets
courses = list(df_courses['Course'])
demand = list(df_course_demand['Size'])
curricula = list(df_curricula['Code'])
timeslots = [i for i in range(9)]
days = ['M_O', 'T_O', 'W_O', 'TH_O', 'F_O'] #Days for odd week
rooms = list(df_rooms['ROOM INDEX'])
room_type = [1,2,3]
#Lectures can happen in Theatre, Classroom
#Workshops happen in Teaching Studio, Classroom, Boardroom 
class_type = ['L', 'W']
workshop_pattern = [1, 2, 3]

capacities = sorted(np.append(df_rooms['CAP'].unique(), 0))
# Parameters 
lecture_hours = list(df_courses['Lecture - Duration'])
workshop_hours = list(df_courses['Workshop - Duration'])



# Create the variables
x = np.array([xp.var( name='x_{0}_{1}_{2}'.format(i, j, k), vartype=xp.binary)
                  for i in courses for j in timeslots for k in class_type], dtype=xp.npvar).reshape(len(courses), len(timeslots), len(class_type))

y = np.array([xp.var( name='y_{0}_{1}_{2}'.format(i, j, s), vartype=xp.binary)
                  for i in courses for j in timeslots for s in capacities], dtype=xp.npvar).reshape(len(courses), len(timeslots), len(capacities))

z = np.array([xp.var( name='z_{0}_{1}'.format(i, d), vartype=xp.binary)
                  for i in courses for d in days], dtype=xp.npvar).reshape(len(courses), len(days))

w = np.array([xp.var( name='w_{0}'.format(i))
                    for i in courses], dtype=xp.npvar)

r = np.array([xp.var( name='r_{0}_{1}'.format(cu, j), vartype=xp.binary)
                  for  cu in curricula for j in timeslots], dtype=xp.npvar).reshape(len(curricula), len(timeslots))

v = np.array([xp.var( name='v_{0}_{1}'.format(cu, j), vartype=xp.binary)
                  for cu in curricula for j in timeslots], dtype=xp.npvar).reshape(len(curricula), len(timeslots))

#counts number of clashes 
#h = 



In [27]:
# Add the variables to the problem
stage_1.addVariable(x, y, z, w, r, v)


?1030 Error: Duplicate column names are not allowed - column 810 would have same name as column 22302


SolverError: ?1030 Error: Duplicate column names are not allowed - column 810 would have same name as column 22302

In [60]:
type(workshop_hours[0])
workshop_hours

[2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 2,
 2,
 1,
 0,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 3]

In [64]:
#Add constraints

#C1: A course i in C should be assigned exactly L(c) lectures that is L(c)== #timeslots(c)
C1_L = [xp.Sum(x[i, j, 0] for j in (timeslots)) == lecture_hours[i] for i in range(len(courses))]
#for workshops
C1_W = [xp.Sum(x[i, j, 1] for j in (timeslots)) == workshop_hours[i] for i in range(len(courses))]

#C2: At s given time period we do not assign more courses than available rooms in general for both types

#!!! separate the types
C2 = [xp.Sum(x[i, j, k] for i in range(len(courses)) for k in range(len(class_type))) <= len(rooms) for j in (timeslots)]

# Lectures on types 1, 3
#C2_L = [x[i, j, 0] == 0 for i in range(len(courses)) for j in timeslots]
# Workshops on types 2, 3
#C2_W = [x[i, j, 0] == 0 for i in range(len(courses)) for j in timeslots]

#C3: Makes sure that if a room is assigned to a course in a time period, then course is assigned to that time period
C3 = [x[i,j,k] - y[i,j,s] >= 0 for i in range(len(courses)) for j in (timeslots) for k in range(len(class_type)) for s in range(len(capacities))]

#C4: ?

#C5: Calculates if classtypes from a course are planned on a day 
C5 = [xp.Sum(x[i,j,k] for j in (timeslots)) - z[i,d] >= 0 for i in range(len(courses)) for k in (range(len(class_type))) for d in range(len((days)))]

#C6: Violation of mnd(c)
#C6 = 

#C7: Only one course from curriculum is planned at timeslot t
#NEED COURSES IN CURRICULA FOR THIS 
#C7 = [xp.Sum(x[i,j,k]-r[cu, j] for c in range(len(curricula))) == 0 for cu in range(len(curricula)) for j in (timeslots)]



#C8: Calculates curriculum compactness violation 
#C8 = [-r[cu,j-1] + r[cu,j] - r[cu, j+1] - v[cu,j] <= 0 for cu in range(len(curricula)) for j in [1,2,3,4,5,6,7]]

#C9: Demand is satisfied for classes

C9 = [y[i, j, s] == 0 for i in range(len(courses)) for j in timeslots for s in range(len(capacities)) if s < demand[i]]

In [ ]:
#Add constraints 
#stage_1.addConstraint(C1, )


In [ ]:
#Write objective function 
stage_1.setObjective(x + y, sense = xp.minimize)